# Gradient-Based Post Training Quantization using the Model Compression Toolkit - A Quick-Start Guide

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/mct_features_notebooks/keras/example_keras_mobilenet_gptq.ipynb)

## Overview

This tutorial demonstrates a pre-trained model quantization using the **Model Compression Toolkit (MCT)** with **Gradient-based PTQ (GPTQ)**. 
GPTQ stands as an optimization procedure that markedly enhances the performance of models undergoing post-training quantization.
This is achieved through an optimization process applied post-quantization, specifically adjusting the rounding of quantized weights.
GPTQ is especially effective in case of low bit width quantization and mixed precision quantization. 

This tutorial's scope is limited to demonstrating GPTQ usage. In this example, we quantize the model and evaluate the accuracy before and after quantization.

For an example of a full quantization flow utilizing GPTQ see [full quantization tutorial](https://github.com/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/keras/keras_yolov8n_for_imx500.ipynb)

## Summary

In this tutorial we will cover:

1. Gradient-Based Post-Training Quantization using MCT.
2. Loading and preprocessing ImageNet's validation dataset.
3. Constructing an unlabeled representative dataset.
4. Accuracy evaluation of the floating-point and the quantized models.

## Setup

Install and import the relevant packages:

In [ ]:
TF_VER = '2.14.0'

!pip install -q tensorflow[and-cuda]~={TF_VER}

In [ ]:
import importlib
if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

In [ ]:
import tensorflow as tf
import keras
import importlib.util

Load a pre-trained MobileNetV2 model from Keras, in 32-bits floating-point precision format.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

float_model = MobileNetV2()

## Dataset preparation
### Download the ImageNet validation set
Download the ImageNet dataset with only the validation split.
**Note:** For demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [ ]:
import os
 
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    
    !cd imagenet && tar -xzf ILSVRC2012_devkit_t12.tar.gz && \
     mkdir ILSVRC2012_img_val && tar -xf ILSVRC2012_img_val.tar -C ILSVRC2012_img_val

The following code organizes the extracted data into separate folders for each label, making it compatible with Keras dataset loaders.

In [ ]:
from pathlib import Path
import shutil

root = Path('./imagenet')
imgs_dir = root / 'ILSVRC2012_img_val'
target_dir = root /'val'

def extract_labels():
    !pip install -q scipy
    import scipy
    mat = scipy.io.loadmat(root / 'ILSVRC2012_devkit_t12/data/meta.mat', squeeze_me=True)
    cls_to_nid = {s[0]: s[1] for i, s in enumerate(mat['synsets']) if s[4] == 0} 
    with open(root / 'ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt', 'r') as f:
        return [cls_to_nid[int(cls)] for cls in f.readlines()]

if not target_dir.exists():
    labels = extract_labels()
    for lbl in set(labels):
        os.makedirs(target_dir / lbl)
    
    for img_file, lbl in zip(sorted(os.listdir(imgs_dir)), labels):
        shutil.move(imgs_dir / img_file, target_dir / lbl)


These functions generate a `tf.data.Dataset` from image files in a directory.

In [ ]:
def imagenet_preprocess_input(images, labels):
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

def get_dataset(batch_size, shuffle):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory='./imagenet/val',
        batch_size=batch_size,
        image_size=[224, 224],
        shuffle=shuffle,
        crop_to_aspect_ratio=True,
        interpolation='bilinear')
    dataset = dataset.map(lambda x, y: (imagenet_preprocess_input(x, y)), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## Representative Dataset

GPTQ is a gradient-based optimization process, which requires representative dataset to perform inference and compute gradients. 

Separate representative datasets can be used for the PTQ statistics collection and for GPTQ. In this tutorial we use the same representative dataset for both.

A complete pass through the representative dataset generator constitutes an epoch (batch_size x n_iter samples). 

In [ ]:
batch_size = 16
n_iter = 5

dataset = get_dataset(batch_size, shuffle=True)

def representative_dataset_gen():
    for _ in range(n_iter):
        yield [dataset.take(1).get_single_element()[0].numpy()]

## Model Gradient-Based Post-Training Quantization using MCT

This is the main part in which we quantize and our model.
First, we create a GPTQ configuration with possible GPTQ optimization options (such as the number of epochs for the optimization process). MCT will quantize the model and start the GPTQ process to optimize the model’s parameters and quantization parameters.

In addition, we need to define a TargetPlatformCapability object, representing the HW specifications on which we wish to eventually deploy our quantized model.

In [ ]:
import model_compression_toolkit as mct

# Create a GPTQ quantization configuration and set the number of training iterations. 
# 50 epochs are sufficient for this tutorial. For GPTQ run after mixed precision quantization a higher number of iterations
# will be required.
gptq_config = mct.gptq.get_keras_gptq_config(n_epochs=50)

# Specify the target platform capability (TPC)
tpc = mct.get_target_platform_capabilities("tensorflow", 'imx500', target_platform_version='v1')


### Run model Gradient-based Post-Training Quantization
Finally, we quantize our model using MCT's GPTQ API.

In [ ]:
quantized_model, quantization_info = mct.gptq.keras_gradient_post_training_quantization(
    float_model,
    representative_dataset_gen,
    gptq_config=gptq_config,
    target_platform_capabilities=tpc)

That's it! Our model is now quantized.

## Model Evaluation

In order to evaluate our models, we first need to load the validation dataset.

In [ ]:
val_dataset = get_dataset(batch_size=50, shuffle=False)

Let's start with the floating-point model evaluation. We need to compile the model before evaluation and set the loss and the evaluation metric.

In [ ]:
float_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
float_accuracy = float_model.evaluate(val_dataset)
print(f"Float model's Top 1 accuracy on the Imagenet validation set: {(float_accuracy[1] * 100):.2f}%")

Finally, let's evaluate the quantized model:

In [ ]:
quantized_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
quantized_accuracy = float_model.evaluate(val_dataset)
print(f"Quantized model's Top 1 accuracy on the Imagenet validation set: {(quantized_accuracy[1] * 100):.2f}%")

You can see that we got a very small degradation with a compression rate of x4 !
Now, we can export the model to Keras and TFLite:

In [ ]:
mct.exporter.keras_export_model(model=quantized_model, save_model_path='qmodel.tflite',
                                serialization_format=mct.exporter.KerasExportSerializationFormat.TFLITE, quantization_format=mct.exporter.QuantizationFormat.FAKELY_QUANT)

mct.exporter.keras_export_model(model=quantized_model, save_model_path='qmodel.keras')


## Conclusion
In this tutorial, we demonstrated how to quantize a pre-trained model using MCT with gradient-based optimization with a few lines of code. We saw that we can achieve an x4 compression ratio with minimal performance degradation.




Copyright 2023 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
